In [1]:
import soundfile as sf
from pathlib import Path
import librosa
from os import (listdir,remove)
import pandas as pd
import numpy as np

In [3]:
base_dir=Path("E:\Cornell_Bird_Call_Noise")

In [35]:
def convert_wav():
    folders=[f for f in listdir(base_dir)]
    for folder in folders:
        sub_dir=base_dir / folder
        files=[f for f in listdir(sub_dir)]
        for f in files:
            print("converting and resampling file {} to 32k sr wav file".format(f))
            y, _ = librosa.load(
                sub_dir / f ,
                sr=32000, 
                mono=True, 
                res_type="kaiser_fast"
            )
            f_list=f.split('.')
            f_list[1]=".wav"
            f=''.join(f_list)
            sf.write(sub_dir / f, y, samplerate=32000)


convert_wav()      

converting and resampling file 342462__nick121087__morning-birds.wav to 32k sr wav file
converting and resampling file 345313__embracetheart__city-traffic-outdoor.wav to 32k sr wav file
converting and resampling file 44796__gezortenplotz__nyc-street-leve02l.wav to 32k sr wav file
converting and resampling file 163597__naturesoundspa__nature-sound-spa-charming-birds-in-a-forest.mp3 to 32k sr wav file


e:\vans\envs\deeplearning\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


converting and resampling file 27221__ls__rain.wav to 32k sr wav file
converting and resampling file 398980__inspectorj__ambience-day-wildlife-a.wav to 32k sr wav file
converting and resampling file 434886__klankbeeld__pinewoods-july-nl-giersbergen-short-180715-1270.wav to 32k sr wav file
converting and resampling file 469009__innorecords__relaxing-mountains-rivers-streams-running-water.wav to 32k sr wav file
converting and resampling file 476202__bohemiancelticgal__rain-drips.mp3 to 32k sr wav file
converting and resampling file 448213__florianreichelt__white-noise.wav to 32k sr wav file


In [36]:
sanity_check_audio,sr=sf.read(Path(base_dir/ "City" / "44796__gezortenplotz__nyc-street-leve02l.wav"))


In [41]:
sanity_check_audio.shape

(788550,)

In [42]:
def convert_5_seconds():
    folders=[f for f in listdir(base_dir)]
    dit={"file_name":[],"type":[]}
    final_dir=base_dir / "10_sec_clip"
    if  not final_dir.exists():
        final_dir.mkdir(parents=True, exist_ok=True)
    for folder in folders:
        sub_dir=base_dir / folder
        files=[f for f in listdir(sub_dir)]
        count=-1
        for f in files:
            y, sr = sf.read(sub_dir / f)
            print(f)
            print(sr)
            print(y.shape)
            count+=1
            for i in range(0,len(y),10*sr):
                f_list=f.split('.')
                f_list[0]=str(folder)+"_"+str(count)+"_"+str(i//(10*sr))+'.'
                f_exit=''.join(f_list)
                dit["file_name"].append(f_exit)
                dit["type"].append(str(folder))
                z=y[i:i+10*sr]
                try:
                    z=z[:,0]
                except:
                    pass
                print(z.shape)
                sf.write(final_dir / f_exit,z,samplerate=32000)
    dit=pd.DataFrame(dit)
    dit.to_csv(final_dir / "df.csv")
    print("csv generated")
    return dit
                
df=convert_5_seconds()           

342462__nick121087__morning-birds.wav
32000
(3198408,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(318408,)
345313__embracetheart__city-traffic-outdoor.wav
32000
(1301804,)
(320000,)
(320000,)
(320000,)
(320000,)
(21804,)
44796__gezortenplotz__nyc-street-leve02l.wav
32000
(788550,)
(320000,)
(320000,)
(148550,)
163597__naturesoundspa__nature-sound-spa-charming-birds-in-a-forest.wav
32000
(969216,)
(320000,)
(320000,)
(320000,)
(9216,)
27221__ls__rain.wav
32000
(799243,)
(320000,)
(320000,)
(159243,)
398980__inspectorj__ambience-day-wildlife-a.wav
32000
(1909226,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(309226,)
434886__klankbeeld__pinewoods-july-nl-giersbergen-short-180715-1270.wav
32000
(11296480,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(320000,)
(32

In [43]:
print(df)

               file_name         type
0          Birds_0_0.wav        Birds
1          Birds_0_1.wav        Birds
2          Birds_0_2.wav        Birds
3          Birds_0_3.wav        Birds
4          Birds_0_4.wav        Birds
..                   ...          ...
97        Nature_5_0.wav       Nature
98        Nature_5_1.wav       Nature
99   White_noise_0_0.wav  White_noise
100  White_noise_0_1.wav  White_noise
101  White_noise_0_2.wav  White_noise

[102 rows x 2 columns]


In [44]:
import soundfile as sf
import numpy as np
import pandas as pd
import random



def add_noise(audio,coeff):
    ## input -> audio file wihtout noise , coeff is the proportion of noise to be added
    ## output ->audio file with noise
    noise_dir=Path(r"E:\Cornell_Bird_Call_Noise\10_sec_clip")
    noise_df=pd.read_csv(Path(noise_dir / "df.csv"))
    samp=np.random.randint(len(noise_df))
    noise_file_path=noise_dir / noise_df.iloc[samp].file_name
    print(noise_file_path)
    temp=noise_file_path
    n_y, n_sr = sf.read(noise_file_path)
    try:
        n_y=n_y[:,0]
    except:
        pass
    length = n_y.shape[0]
    effective_length = sr * 5
    if length < effective_length:
        new_y = np.zeros(effective_length, dtype=n_y.dtype)
        start = np.random.randint(effective_length - length)
        new_y[start:start + length] = n_y
        n_y = new_y.astype(np.float32)
    elif length > effective_length:
        start = np.random.randint(length - effective_length)
        n_y = n_y[start:start + effective_length].astype(np.float32)
    else:
        n_y = n_y.astype(np.float32)
    noise_energy = np.sqrt(n_y.dot(n_y))
    audio_energy = np.sqrt(audio.dot(audio))
    print(noise_energy)
    print(audio_energy)
    audio += coeff * n_y *( audio_energy / noise_energy)
    return audio,n_sr

In [49]:
audio_file=r"C:\Users\M AMIN\Downloads\audio_without_noise\datasets_756953_1307546_hamfly_XC123913.wav"
y,sr=sf.read(audio_file)
length = y.shape[0]
effective_length = sr * 5

if length < effective_length:
    new_y = np.zeros(effective_length, dtype=y.dtype)
    start = np.random.randint(effective_length - length)
    new_y[start:start + length] = y
    y = new_y.astype(np.float32)
elif length > effective_length:
    start = np.random.randint(length - effective_length)
    y = y[start:start + effective_length].astype(np.float32)
else:
    y = y.astype(np.float32)
y_n,_=add_noise(y,0.7)

E:\Cornell_Bird_Call_Noise\10_sec_clip\Nature_4_8.wav
1.6895278
0.31746867


In [50]:
import IPython.display as ipd
print(y_n.shape)
sf.write(r"C:\Users\M AMIN\Downloads\audio_without_noise\test.wav", y_n, samplerate=32000)

(160000,)


In [52]:
ipd.Audio(r"C:\Users\M AMIN\Downloads\audio_without_noise\test.wav") 

In [ ]:
ipd.Audio(r"E:\Cornell_Bird_Call_Noise\10_sec_clip\Nature_3_34.wav") 